# Implementation of Canny Edge detector Algorithm

In [ ]:
import numpy as np
import os
import cv2
from scipy import ndimage


def Canny_detector(img,std=1,kernel_size=7,lowthreshold=20,highthreshold=50):
  
  #Gaussian Kernel
  x=kernel_size//2
  d_kernel_linespace = arange(-x,x+1)
  hkernel=[np.exp(-i**2/2*std**2) for i in d_kernel_linespace]
  kernel_sum = np.sum(hkernel)
  hkernel    = hkernel/kernel_sum
  #vkernel=np.vstack(hkernel)
  kernel = [[x*y for x in hkernel] for y in hkernel]
  kernel = np.array(kernel)
  
   #Turning image into gray
  def one_channel(img):
    x = np.sum(img,axis=2)
    return x//3
    
  gray_image = one_channel(img)
  blurred_img = ndimage.filters.convolve(gray_image, kernel) #denoising image
  blurred_img=np.uint8(blurred_img) 
  
  
  #taking derivative of the image using sobel filters
  kx=np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
  ky=np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
  #x= ndimage.filters.convolve(blurred_imgs[0],kx)
  #y=ndimage.filters.convolve(blurred_imgs[0],ky)
  sobel_x=cv2.filter2D(blurred_img, -1, kx)
  sobel_y=cv2.filter2D(blurred_img, -1, ky)  
  grad=np.hypot(sobel_x,sobel_y)
  grad=grad/grad.max() *255
  gradient=np.uint8(grad)
  radiant = np.arctan2(sobel_y,sobel_x)
  
  
  
  grad = gradient.copy()
  rows,columns = grad.shape
  theta = (radiant *180)/(np.pi)
  mask = theta<0
  theta[mask] += 180

    
    
  #Non Maxima supression in the four directions 0,45,90,135
  for r in range(1,rows-1):

    for c in range(1,columns-1):
 
      if (0 <= theta[r,c] <= 22.5) or (157.5 <= theta[r,c] <= 180):
        if (grad[r,c] <= grad[r,c-1]) or (grad[r,c] <= grad[r,c+1]):
          grad[r,c]=0
      

      elif (22.5 < theta[r,c] <= 67.5):
        if (grad[r,c] <= grad[r-1,c+1]) or (grad[r,c] <= grad[r+1,c-1]):
          grad[r,c]=0
          
          
          
      elif (67.5 < theta[r,c] <= 112.5):
        if grad[r,c] <= grad[r-1,c] or grad[r,c] <= grad[r+1,c]:
          grad[r,c]=0
          
          
          
      elif (112.5 < theta[r,c] < 157.5):
        if grad[r,c] <= grad[r-1,c-1] or grad[r,c] <= grad[r+1,c+1]:
          grad[r,c]=0
          
          
          

  #strong and week edges step
  strong_edges = np.array((grad>highthreshold),dtype='uint8')
  week_edges   = np.array((grad>lowthreshold),dtype='uint8')
  threshold_edges = strong_edges + week_edges  

  
  rows,columns = threshold_edges.shape
  
  #Edge tracking by hysteresis , Here i use the image as a graph and then apply DFS algorithm to track strong edges
  for r in range(1,rows-1):
    for c in range(1,columns-1):
      
      if (threshold_edges[r,c] == 2):
        stack = [(r,c)]
        while len(stack)>0:
          current_pixel = stack.pop(0)
          x,y           = current_pixel
          threshold_edges[x,y] = 2
          
          try:
            if threshold_edges[x,y-1] == 1:
              stack.insert(0,(x,y-1))
              
            if threshold_edges[x,y+1] == 1:
              stack.insert(0,(x,y+1))
            
            if threshold_edges[x-1,y] == 1:
              stack.insert(0,(x-1,y))
              
            if threshold_edges[x+1,y] == 1:
              stack.insert(0,(x+1,y))
              
            if threshold_edges[x-1,y-1] == 1:
              stack.insert(0,(x-1,y-1))
            
            if threshold_edges[x+1,y-1] == 1:
              stack.insert(0,(x+1,y-1))
              
            if threshold_edges[x-1,y+1] == 1:
              stack.insert(0,(x-1,y+1))
              
            if threshold_edges[x+1,y+1] == 1:
              stack.insert(0,(x+1,y+1))
              
          except:
            pass  
          
  grad[threshold_edges==2]=255
  grad[threshold_edges!=2]=0
  return grad